In [ ]:
import tensorflow as tf
import numpy as np
from scipy.misc import imsave

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data')

x = tf.placeholder(tf.float32, shape=[None, 784])

x_img = tf.reshape(x, [-1, 28, 28, 1])

conv1 = tf.layers.conv2d(
    inputs=x_img,
    filters=8,
    kernel_size=[3, 3],
    padding="SAME",
    activation=tf.nn.relu)

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=4,
    kernel_size=[3, 3],
    padding="SAME",
    activation=tf.nn.relu)

pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

conv3 = tf.layers.conv2d(
    inputs=pool2,
    filters=2,
    kernel_size=[3, 3],
    padding="SAME",
    activation=tf.nn.relu)

conv_t1 = tf.layers.conv2d_transpose(
    inputs=conv3,
    filters=4,
    kernel_size=(2, 2),
    strides=(2, 2),
    padding='SAME')

conv4 = tf.layers.conv2d(
    inputs=conv_t1,
    filters=4,
    kernel_size=[3, 3],
    padding="SAME",
    activation=tf.nn.relu)

conv_t2 = tf.layers.conv2d_transpose(
    inputs=conv4,
    filters=8,
    kernel_size=(2, 2),
    strides=(2, 2),
    padding='SAME')

conv5 = tf.layers.conv2d(
    inputs=conv_t2,
    filters=8,
    kernel_size=[3, 3],
    padding="SAME",
    activation=tf.nn.relu)

out_img = tf.layers.conv2d(
    inputs=conv5,
    filters=1,
    kernel_size=[1, 1],
    padding="SAME",
    activation=tf.nn.relu)

loss = tf.reduce_mean(tf.square(out_img - x_img))
tf.summary.scalar('loss', loss)
learning_rate = 0.001
trainer = tf.train.AdamOptimizer(learning_rate)
train_step = trainer.minimize(loss)
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(str(learning_rate), graph=sess.graph)
    for i in range(40000):
        train_batch = mnist.train.next_batch(64)
        train_step.run(feed_dict={x: train_batch[0]})
        if i % 50 == 0:
            print("{}\t{:.6f}".format(i, loss.eval(feed_dict={x: mnist.validation.images})))
            summary_str = sess.run(merged_summary_op, feed_dict={x: mnist.validation.images})
            writer.add_summary(summary_str, i)
    # save visualised results
    n = 0
    if n > 0:
        res_inp = mnist.test.images[10:10+n]
        res_out = np.clip(out_img.eval(feed_dict={x: res_inp}), 0.0, 1.0)
        for i in range(n):
            imsave("in{}.png".format(i), res_inp[i].reshape((28, 28)))
            imsave("out{}.png".format(i), res_out[i].reshape((28, 28)))
    # test loss
    print("{:.6f}".format(loss.eval(feed_dict={x: mnist.test.images})))